# 数字検出・数独ソルバー

___
## ライブラリのインポート・数独ソルバー関数の定義

In [15]:
!pip install pyocr
import copy
import sys
import pyocr
import pyocr.builders
import numpy as np
import glob

import struct
import cv2
import numpy as np
import matplotlib.pyplot as plt 
from IPython.display import Image, display


def sudoku_solve(problem):
    rows, cols = problem.shape
    all_nums = set([i for i in range(0, 10)])
    for i in range(rows):
        for j in range(cols):
            if problem[i, j] == 0:
                # 候補の数字を調べる (以下のnumsに代入する)
                k = 3 * (i // 3)
                l = 3 * (j // 3)
                blk_nums = problem[k:k+3, l:l+3]
                row_nums = problem[i,:]
                col_nums = problem[:,j]
                used_nums = set(row_nums.tolist() + col_nums.tolist() + blk_nums.flatten().tolist())
                nums = set([i for i in range(10)]).difference(used_nums)

                # 候補の数字を仮入れして再帰呼び出し
                success = False
                for n in nums:
                    # 数字を代入
                    problem[i, j] = n
                    if sudoku_solve(problem):
                        success = True
                        break

                    # 失敗したら元に戻す
                    problem[i, j] = 0

                if not success:
                    return False
        
    return True

You should consider upgrading via the '/Users/shota/opt/miniconda3/bin/python -m pip install --upgrade pip' command.


___
## tesseractを用いた数字の検出

In [17]:
from PIL import Image

tools = pyocr.get_available_tools()
if len(tools) == 0:
    print("No OCR tool found")
    sys.exit(1)

tool = tools[0]
lang = 'eng'

row_list = []
res_list = []

split_data = sorted(glob.glob("output/*.png"))

for x in range(1, len(split_data)+1):
    img = cv2.imread(split_data[x-1])
    text = tool.image_to_string(
    #Image.open(split_data[x-1]),
    Image.fromarray(img),
    lang=lang,
    # builder=pyocr.builders.DigitBuilder()
    builder=pyocr.builders.DigitBuilder(tesseract_layout=6)
    )
    if text == "":
        row_list.append("0")
    else:
        row_list.append(text)
    if x%9 == 0:
        res_list.append(copy.deepcopy(row_list))
        row_list = []

problem = [[int(x) for x in y] for y in res_list]
problem = np.array(problem)
print(problem)

[[0 3 5 0 9 0 0 4 8]
 [0 0 9 0 0 8 0 0 3]
 [0 4 0 6 0 5 0 0 1]
 [0 0 0 0 7 4 0 0 0]
 [0 2 0 0 0 0 0 6 0]
 [0 0 0 0 5 0 0 0 0]
 [3 0 0 9 0 2 0 7 0]
 [9 0 0 5 0 0 2 0 0]
 [6 1 0 0 4 0 5 3 0]]


___
## ソルバー関数で数独を解く

In [18]:
sudoku_solve(problem)
print(problem)

[[2 3 5 7 9 1 6 4 8]
 [1 6 9 4 2 8 7 5 3]
 [8 4 7 6 3 5 9 2 1]
 [5 8 6 1 7 4 3 9 2]
 [7 2 1 3 8 9 4 6 5]
 [4 9 3 2 5 6 8 1 7]
 [3 5 8 9 6 2 1 7 4]
 [9 7 4 5 1 3 2 8 6]
 [6 1 2 8 4 7 5 3 9]]
